Clase 6
====
Hoy veremos herramientas avanzadas de cobrapy.

Objetivos

* Lectura de modelos metabólicos
* Flux Varional Analysis (FVA)
* Parsimonious FBA


Lectura de modelos metabólicos
===========================

Cobrapy puede leer y escribir modelos metabólicos en varios formatos, incluyendo SBML, JSON, YAML, MAT, y pickle. Hoy trabajaremos con SBML, el cual es el formato preferido. 

Los modelos metabólicos para varias especies pueden ser descargados [aquí](http://systemsbiology.ucsd.edu/Downloads). Para los ejercicios de hoy descarga el modelo [iMM904 de Saccharomyces cerevisiae](https://raw.githubusercontent.com/modcommet/Clases/master/iMM904.xml)


In [1]:
import cobra
model = cobra.io.read_sbml_model("iMM904.xml") #importa el formato y lo guarda en model
model

Name,iMM904
Memory address,0x05f9d2b0
Number of metabolites,1226
Number of reactions,1577
Objective expression,-1.0*BIOMASS_SC5_notrace_reverse_93090 + 1.0*BIOMASS_SC5_notrace
Compartments,"cytosol, extracellular space, golgi apparatus, mitochondria, nucleus, endoplasmic reticulum, vacuole, peroxisome/glyoxysome"


Running Flux Vartional Analysis (FVA)
=====================================

En general FBA no tiene una solución única, pero varias que resultan en el mismo valor de la función objetivo. El análisis de variabilidad de flujo (FVA: Flux Variational Analysis) permite encontra rel rango de flujos de cada metabólito en el valor optimo.

In [2]:
# Primero encontramos la solución optima del modelo
solution = model.optimize()

In [3]:
# Luego computamos los rangos de flujo de cada metabólito
from cobra.flux_analysis import flux_variability_analysis

fva=flux_variability_analysis(model)

In [4]:
# Para ver los rangos de los primeros 10 metabolitos
fva[0:10]

,maximum,minimum
13BGH,6.974756e-10,0.000000e+00
13BGHe,0.000000e+00,0.000000e+00
13GS,3.266700e-01,3.266700e-01
16GS,0.000000e+00,0.000000e+00
23CAPPD,0.000000e+00,0.000000e+00
2DDA7Ptm,0.000000e+00,-7.608291e-02
2DHPtm,4.123008e-10,0.000000e+00
2DOXG6PP,0.000000e+00,0.000000e+00
2HBO,0.000000e+00,-1.186815e-14
2HBt2,0.000000e+00,-1.186815e-14


parsimonious FBA
=========

Por otra parte el valor optimo encontrado por FBA no necesariamente implica que se están ocupando el minimo de flujos disponibles. Por ejemplo, cuando hay ciclos es posible que algunos flujos tomen valores arbitrariamente altos sin afectar el valor de la función objetivo. Esto puede ser una causa de artefactos. Para prevenir esto se puede usar la herramienta parsimoniouos FBA (pFBA), la cual realiza una optimización en dos niveles. En el nivel interna se optimiza la función objetivo, tal como en un FBA normal, mientras que en la optimización externa se minimiza la suma total de los flujos con la restricción de que se conserve el valor óptimo encontrado por la optimización interna.

Veamos un ejemplo:

In [8]:
# Este es un FBA regular #al tener soluciones alternativas, no todas son validas
fba_solution = model.optimize() #optimizacion interna
# Ahora veamos como seria el pFBA
pfba_solution = cobra.flux_analysis.pfba(model)

In [12]:
# Los valores de las funciones objetivos son:
print fba_solution.f #FBA regular, valor obj FBA
print pfba_solution.f #suma de los flujos

0.287865703704
338.005991582


In [16]:
# Los valores de la reaccion de biomasa (funcion objetivo del FBA) deberia ser el mismo en ambos casos
print fba_solution.fluxes["BIOMASS_SC5_notrace"]
print pfba_solution.fluxes["BIOMASS_SC5_notrace"]
print abs(fba_solution.fluxes["BIOMASS_SC5_notrace"] - pfba_solution.fluxes["BIOMASS_SC5_notrace"])

0.287865703704
0.287865703704
3.88578058619e-16


In [15]:
# Sin embargo la suma de los flujos deberia ser menor en pFBA
print sum(fba_solution.fluxes)
print sum(pfba_solution.fluxes)
fba_solution.fluxes[0:10]

134.787892903
134.463494724


13BGH       0.000000
13BGHe      0.000000
13GS        0.326670
16GS        0.000000
23CAPPD     0.000000
2DDA7Ptm   -0.076083
2DHPtm      0.000000
2DOXG6PP    0.000000
2HBO        0.000000
2HBt2       0.000000
Name: fluxes, dtype: float64

Ejercicio
=========

1. Descargen una la read metabólica de Ecoli 
2. Escriban una función para realizar todos los análsis vistos en este clase
3. Escriban su función como un script.

In [9]:
import cobra
model = cobra.io.read_sbml_model("Ec_iJR904.xml") #importa el formato y lo guarda en model
model

ImportError: libSBML required for fbc < 2

In [3]:
# Primero encontramos la solución optima del modelo
solution = model.optimize()

In [4]:
# Luego computamos los rangos de flujo de cada metabólito
from cobra.flux_analysis import flux_variability_analysis

fva=flux_variability_analysis(model)

In [5]:
# Para ver los rangos de los primeros 10 metabolitos
fva[10:20]

,maximum,minimum
2HMHMBQMTm,7.990703e-11,0.000000e+00
2HP6MPMOm,7.990703e-11,0.000000e+00
2HPMBQMTm,7.990703e-11,0.000000e+00
2HPMMBQMOm,7.990703e-11,0.000000e+00
2MBACt,3.811879e-10,0.000000e+00
2MBALDt,5.709195e-10,0.000000e+00
2MBALDtm,6.008248e-10,0.000000e+00
2MBTOHt,6.008248e-10,0.000000e+00
2MBTOHtm,0.000000e+00,4.272533e-14
2MPPALt,2.684536e-09,0.000000e+00
